In [15]:

train_data_dir = "skin/train"
test_data_dir = "skin/test"

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values for test data

# Set the batch size
batch_size = 16

# Create training and testing data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)


Found 7452 images belonging to 6 classes.
Found 1644 images belonging to 6 classes.


In [17]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2


# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),# Pooling layer
    Dropout(0.5),
    layers.Dense(128, activation='relu'),  # Additional dense layer (adjust as needed)
    layers.Dense(6, activation='softmax')  # 23 disease classes + 1 for normal
])


from tensorflow.keras.optimizers import Adam

# Change the learning rate to a different value, e.g., 0.001
optimizer = Adam(learning_rate=0.001)

# Compile the model with the updated optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # View the model architecture


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               163968    
                                                                 
 dense_5 (Dense)             (None, 6)                 774       
                                                                 
Total params: 2,422,726
Trainable params: 2,388,614
No

In [18]:
# Set the number of training epochs
epochs = 30
# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)


Epoch 1/30
177/466 [==========>...................] - ETA: 1:22 - loss: 1.2148 - accuracy: 0.4862

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

103/103 [==============================] - 8s 79ms/step - loss: 1.0092 - accuracy: 0.7518
Test accuracy: 0.7518247961997986


In [ ]:
# from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import os


# Get the uploaded image file name (assuming only one file is uploaded)
uploaded_image_filename = r"D:\skindisease\skin\test\Eczema Photos\eczema-nummular-11.jpg"



# Get the class labels from the subdirectories in the training data folder
class_labels =  ["Acne and Rosacea Photos","Eczema Photos","Normal","Psoriasis pictures Lichen Planus and related diseases","Tinea Ringworm Candidiasis and other Fungal Infections","vitiligo"]
# print(class_labels)
# Load and preprocess the uploaded image
img = image.load_img(uploaded_image_filename, target_size=(224, 224))  # Resize to (224, 224)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label using the extracted class labels
predicted_class_label = class_labels[predicted_class_index]

# Get the probability of the predicted class
predicted_probability = predictions[0][predicted_class_index]

# Display the results
print(f'Predicted class: {predicted_class_label}')

print(f'Probability: {predicted_probability:.2f}')



NameError: name 'model' is not defined

In [ ]:
model.save('final.h5')
model.save('my_model.keras')

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
loaded_model = tf.keras.models.load_model("final_model_62.h5")

uploaded_image_filename = r"acne.jpg"

# Get the class labels from the subdirectories in the training data folder
class_labels =  ["Acne and Rosacea Photos","Eczema Photos","Normal","Psoriasis pictures Lichen Planus and related diseases","Tinea Ringworm Candidiasis and other Fungal Infections","vitiligo"]
# print(class_labels)
# Load and preprocess the uploaded image
img = image.load_img(uploaded_image_filename, target_size=(224, 224))  # Resize to (224, 224)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label using the extracted class labels
predicted_class_label = class_labels[predicted_class_index]

print(f'Predicted class: {predicted_class_label}')


1/1 [==============================] - 0s 485ms/step
Predicted class: Acne and Rosacea Photos
